<a href="https://colab.research.google.com/github/Danc2050/Distributed-Computing-Scripts/blob/master/google-colab/GoogleColabGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <strong> <h2> Colab CPU Notebook </h2></strong> 
#@markdown <h4>↖️ Click the ▶ button after deciding on the options below.
#@markdown <h4>📜 Please see <a href="https://github.com/Danc2050/Distributed-Computing-Scripts/tree/master/google-colab"> source code</a> for more information.</h4>
#@markdown <h4>🤷🏻‍♂️ Optionally, create a GIMPS/PrimeNet account <a href="https://www.mersenne.org/update/">here.</a></h4>
#@markdown <h4>👷‍♀️ Work Types: </h4>
#@markdown <blockquote> <li> 0 - Whatever makes the most sense </li>
#@markdown  <li> 100 - First time LL tests </li>
#@markdown  <li> 101 - Double-check LL tests </li>
#@markdown  <li> 102 - World record LL tests </li>
#@markdown  <li> 104 - 100 million digit LL tests </li>
#@markdown  <li> 150 - First time PRP tests </li>
#@markdown  <li> 151 - Double-check PRP tests </li>
#@markdown  <li> 152 - World record PRP tests </li>
#@markdown  <li> 154 - 100 million digit PRP tests </li></blockquote>
#@markdown  <h4>🕷️<Strong>Note 1</strong>: 'debug' option shows CUDALucas progress and GPU specs.<h4>
#@markdown <h4>💡 <strong>Note 2</strong>: keep each notebook <strong>open</strong> in a browser to prevent disconnection.<h5>

prime_ID = 'Default' #@param [] {allow-input: true}
prime_password = 'Default' #@param [] {allow-input: true}
CPU_type_of_work = '100' #@param [0, 100, 101, 102, 104, 150, 151, 152, 154]
GPU_type_of_work = '100' #@param [100, 101, 102, 104]
computer_name = 'Default' #@param [] {allow-input: true}
debug = "True (exits after completion)" #@param ["True (exits after completion)", "False"]

def run():
  # Run CUDALucas and Mprime
  !cd cudalucas; nohup python primenet.py -d -T 100 -u prime_ID -p password & # Start PrimeNet to get/submit work. If you wish, replace 'psu' & 'vikings' with your GIMPS Username & Password
  !sleep 1.5
  !cd cudalucas && chmod 777 CUDALucas; nohup ./CUDALucas & # Outputs to nohup.out in cudalucas directory
  !cd mprime_gpu && chmod 777 mprime && ./mprime -d

def install():
  # Download/Install/Configure CUDALucas
  !wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info
  !nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this Notebook
  !wget https://raw.github.com/tdulcet/Distributed-Computing-Scripts/master/cudalucas.sh -q # download CUDALucas script
  !sed -i '/^if ! COMPUTE=/,/^fi/!b; /^fi/a echo "$COMPUTE"' cudalucas.sh # Output CUDA compute capability of GPU
  !sed -i 's/\/$COMPUTE/\/--generate-code arch=compute_35,code=sm_35 --generate-code arch=compute_50,code=sm_50 --generate-code arch=compute_60,code=sm_60 --generate-code arch=compute_70,code=sm_70 --generate-code arch=compute_75,code=sm_75/' cudalucas.sh
  !sed -i '/^GPU=/,/^fi/ s/^/# /' cudalucas.sh # Do not check for an Nvidia GPU
  !sed -i '/^nohup / s/^/# /' cudalucas.sh # Do not start PrimeNet
  !sed -i '/^crontab / s/^/# /' cudalucas.sh # Do not create a cronjob
  !bash -- cudalucas.sh "" # Do not pass any additional parameters here
  run()

  # Download/Install/Configure Prime95
  !wget https://raw.github.com/tdulcet/Distributed-Computing-Scripts/master/mprime.sh -q # Download Prime95
  !rm mprime.sh # remove any old download of the Prime95 script (for case of using 2 notebook maximum)
  !sed -i '/^nohup / s/^/# /' mprime.sh # Do not start Prime95
  !sed -i '/^crontab / s/^/# /' mprime.sh # Do not create a cronjob
  !sed -i 's/"mprime"/"mprime_gpu"/' mprime.sh # Name the folder specific to the runtime type
  !bash -- mprime.sh prime_ID # Replace 'psu' with your own username if you wish and, optionally, add a name for the machine
  run()

def debug_exit():
  !nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this notebook
  !tail -n 100 cudalucas/nohup.out # view CUDALucas progress

# Load & cd into gdrive for persistent data
from google.colab import drive
import os
drive.mount('/content/gdrive')
%cd gdrive/'My Drive'

# use input from user
prime_ID = 'psu' if prime_ID.lower() == 'default' else prime_ID
computer_name = "" if computer_name.lower() == 'default' else computer_name
prime_password = 'vikings' if prime_password.lower() == 'default' else prime_password
debug = False if debug == 'False' else True

# GPU Check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the "Runtime" -> "Change runtime type" -> "GPU" -> "SAVE" to enable a GPU accelerator, ')
    print('and then re-execute this cell.')

if debug:
  print("Debugging")
  debug_exit()

elif os.path.exists('mprime_gpu') and os.path.exists('cudalucas') :
  print("running")
  run()

else:
  print("Installing")
  install()

print("Gracefully exiting...")